In [233]:
import pandas as pd

In [240]:
df_agg_all = pd.read_csv(
    "/Users/robinfeldmann/TopicAnalysisRUWTweets/Topics_10/Topics_0/lang_topic_sentiment_agg.csv"
)
df_agg_all = df_agg_all[df_agg_all["sentiment_label"] != "error"]

In [243]:
df_agg_all

,Unnamed: 0,month,lang,topic,sentiment_label,count
0,0,2022-02,de,0,negative,774
1,1,2022-02,de,0,neutral,871
2,2,2022-02,de,0,positive,25
3,3,2022-02,de,1,negative,545
4,4,2022-02,de,1,neutral,1645
...,...,...,...,...,...,...
3566,3566,2023-06,uk,8,neutral,262
3567,3567,2023-06,uk,8,positive,36
3568,3568,2023-06,uk,9,negative,459
3569,3569,2023-06,uk,9,neutral,1104


In [443]:
def time_table(topic: str) -> pd.DataFrame:
    df_all = (
        df_agg_all.groupby(["month", "sentiment_label"])
        .sum()["count"]
        .reset_index()
        .sort_values("month")
        .pivot(index="sentiment_label", columns="month")
    )
    df_all.columns = df_all.columns.droplevel(0)
    df_all.columns.name = None
    df_all = df_all.reset_index()

    df_cur = (
        df_agg_all.query(f"topic=={topic}")
        .groupby(["month", "sentiment_label"])
        .sum()["count"]
        .reset_index()
        .sort_values("month")
        .pivot(index="sentiment_label", columns="month")
    )
    df_cur.columns = df_cur.columns.droplevel(0)
    df_cur.columns.name = None
    df_cur = df_cur.reset_index()

    all_months = df_agg_all["month"].unique()

    abs = []
    abs_rel = []
    abs_all = []
    for month in all_months:
        abs.append(df_cur[month].sum())
        abs_all.append(df_all[month].sum())
        abs_rel.append(df_cur[month].sum() / df_all[month].sum())

    df_cur.loc[3] = ["rel"] + abs_rel
    df_all.loc[4] = ["total"] + abs_all
    df_cur.loc[4] = ["total"] + abs
    k = pd.DataFrame()

    all_tweets_in_topic = sum(abs)
    all_tweets = sum(abs_all)
    all_neg = df_cur.set_index("sentiment_label").T["negative"].sum()
    all_neut = df_cur.set_index("sentiment_label").T["neutral"].sum()
    all_pos = df_cur.set_index("sentiment_label").T["positive"].sum()

    for month in all_months:
        k[month] = (
            df_cur.set_index("sentiment_label")[month]
            / df_cur.set_index("sentiment_label")[month]["total"]
        )

    k.loc["total"] = abs
    k.loc["rel"] = abs_rel
    k = k.reset_index()
    k["sum"] = df_cur[all_months].sum(axis=1)
    k["sum_rel"] = df_cur[all_months].sum(axis=1) / df_all[all_months].sum(axis=1)
    k["All time"] = [
        all_neg / all_tweets_in_topic,
        all_neut / all_tweets_in_topic,
        all_pos / all_tweets_in_topic,
        all_tweets_in_topic / all_tweets,
        all_tweets_in_topic,
    ]

    return k.set_index("sentiment_label").T


def make_time_table(topic: str):
    df = time_table(topic)

    all_months = df_agg_all.sort_values("month")["month"].unique()

    print(
        "\\begin{table}[H]\n\\centering\n\\hspace*{-1.5cm} \n\\begin{tabular}{||c || c | c | c | c | c ||} "
    )
    print("\\hline")
    header = (
        f"{'Month':10} & {'Negative':>12} & {'Neutral':>12} & {'Positive':>12} & {'Relative Count':>12} & {'Count':>12} "
        + " \\\\ "
    )
    print(header)
    print("\\hline")
    for month in ["All time", *all_months]:

        row = df.loc[month]
        line = (
            f"{month:10} & "
            + " & ".join([f"{val:>12.0%}" for val in row.to_list()[:-1]]).replace(
                "%", "\%"
            )
            + f" & {int(row['total']):>12} "
            + " \\\\"
        )
        #     if i == 4 or i==3:
        #         print("\\hline")
        #     if i == 4:
        #         line = line.replace("%"," ")
        print(line)

    print("\\hline\\hline")
    print("\end{tabular}")
    print(
        f"\\caption{{Number of tweets split by Sentiments and Month for Topic {topic}.}}"
    )
    print(f"\\label{{table:ovtime{topic}}}")
    print("\\end{table}")


make_time_table("9")

\begin{table}[H]
\centering
\hspace*{-1.5cm} 
\begin{tabular}{||c || c | c | c | c | c ||} 
\hline
Month      &     Negative &      Neutral &     Positive & Relative Count &        Count  \\ 
\hline
All time   &          22\% &          48\% &          30\% &          11\% &      2000420  \\
2022-02    &          34\% &          46\% &          20\% &           2\% &         5714  \\
2022-03    &          22\% &          39\% &          39\% &           5\% &       133589  \\
2022-04    &          30\% &          45\% &          25\% &           4\% &        73534  \\
2022-05    &          20\% &          54\% &          26\% &          11\% &       217907  \\
2022-06    &          20\% &          59\% &          21\% &          11\% &       157008  \\
2022-07    &          21\% &          54\% &          25\% &          11\% &       122713  \\
2022-08    &          22\% &          48\% &          30\% &          13\% &       131026  \\
2022-09    &          24\% &          47\% &     

In [447]:
def get_table(topic: str):

    df_pivot_cur = (
        df_agg_all.query(f"topic=={topic}")
        .groupby(["sentiment_label", "lang"])
        .sum()
        .reset_index()[["sentiment_label", "lang", "count"]]
        .pivot(index="sentiment_label", columns="lang")
    )
    df_pivot_cur.columns = df_pivot_cur.columns.droplevel(0)
    df_pivot_cur.columns.name = None
    df_pivot_cur = df_pivot_cur.reset_index()
    df_pivot_cur["sum"] = df_pivot_cur[df_agg_all["lang"].unique()].sum(axis=1)
    df_pivot_cur.loc[3] = (
        ["sum"]
        + [df_pivot_cur[lang].sum() for lang in df_agg_all["lang"].unique()]
        + [df_pivot_cur["sum"].sum()]
    )

    df_pivot_all = (
        df_agg_all.groupby(["sentiment_label", "lang"])
        .sum()
        .reset_index()[["sentiment_label", "lang", "count"]]
        .pivot(index="sentiment_label", columns="lang")
    )
    df_pivot_all.columns = df_pivot_all.columns.droplevel(0)
    df_pivot_all.columns.name = None
    df_pivot_all = df_pivot_all.reset_index()
    df_pivot_all["sum"] = df_pivot_all[df_agg_all["lang"].unique()].sum(axis=1)
    df_pivot_all.loc[3] = (
        ["sum"]
        + [df_pivot_all[lang].sum() for lang in df_agg_all["lang"].unique()]
        + [df_pivot_all["sum"].sum()]
    )
    all_tweets = df_agg_all["count"].sum()
    topic_tweets = (
        df_agg_all.query(f"topic=={topic}").groupby("lang").sum()["count"].sum()
    )
    return pd.DataFrame(
        data=df_pivot_cur.set_index("sentiment_label").values
        / (
            df_pivot_all.set_index("sentiment_label").values
            * (topic_tweets / all_tweets)
        ),
        index=df_pivot_all["sentiment_label"],
        columns=df_pivot_all.columns[1:],
    ).reset_index()


def print_as_table(topic: str):
    df = get_table(topic)
    all_langauges = df_agg_all["lang"].unique()
    language_mapping = {
        "de": "German",
        "en": "English",
        "fr": "French",
        "es": "Spanish",
        "it": "Italian",
        "uk": "Ukrainian",
        "ru": "Russian",
    }

    print(
        "\\begin{table}[H]\n\\centering\n\\hspace*{-1.5cm} \n\\begin{tabular}{||c || c | c | c | c | c | c | c | c ||} "
    )
    print("\\hline")
    header = (
        f"{'Label':10} & "
        + " & ".join([f"{language_mapping[lang]:>10}" for lang in all_langauges])
        + f" & {'All languages':>15}"
        + " \\\\ "
    )
    print(header)
    print("\\hline")
    for a in range(4):
        row = df.iloc[a]
        line = (
            f"{(row['sentiment_label'] if a < 3 else 'all'):10} & "
            + " & ".join(
                [f"{row[k]:10.0%}".replace("%", "\%") for k in [*all_langauges, "sum"]]
            )
            + " \\\\"
        )
        if a == 3:
            print("\\hline")
        print(line)
    print("\\hline\\hline")
    print("\end{tabular}")
    print(
        f"\\caption{{Number of tweets split by Sentiments and Languages for Topic {topic}, relative to their Distribution in the overall Dataset.}}"
    )
    print(f"\\label{{table:ov{topic}}}")
    print("\\end{table}")


print_as_table("0")

\begin{table}[H]
\centering
\hspace*{-1.5cm} 
\begin{tabular}{||c || c | c | c | c | c | c | c | c ||} 
\hline
Label      &     German &    English &    Spanish &     French &    Italian &    Russian &  Ukrainian &   All languages \\ 
\hline
negative   &        42\% &       141\% &       166\% &       117\% &        65\% &       155\% &        68\% &       124\% \\
neutral    &        75\% &        96\% &       112\% &       104\% &        80\% &       100\% &        34\% &        91\% \\
positive   &        12\% &        20\% &        26\% &        40\% &        15\% &        12\% &         4\% &        20\% \\
\hline
all        &        52\% &       111\% &       128\% &       101\% &        64\% &       111\% &        39\% &       100\% \\
\hline\hline
\end{tabular}
\caption{Number of tweets split by Sentiments and Languages for Topic 0, relative to their Distribution in the overall Dataset.}
\label{table:ov0}
\end{table}


In [170]:
df_pivot_cur.set_index("sentiment_label").values / (
    df_pivot_all.set_index("sentiment_label").values * (topic_tweets / all_tweets)
)

array([[0.42281426, 1.40871017, 1.65911364, 1.17234223, 0.65083181,
        1.54716347, 0.67830738, 1.23769892],
       [0.75091744, 0.96418344, 1.11764138, 1.0409536 , 0.80169867,
        0.99774207, 0.34171255, 0.90592132],
       [0.12290874, 0.20454932, 0.25876088, 0.40019286, 0.15239726,
        0.1172504 , 0.04114527, 0.20179369],
       [0.5214832 , 1.11374737, 1.28252407, 1.0145451 , 0.64231419,
        1.11037755, 0.38892293, 1.00000011]])

In [245]:
df_pivot_all

,sentiment_label,de,en,es,fr,it,ru,uk,sum
0,negative,903253,6629371,482790,659712,773145,173005,282081,9903357
1,neutral,571670,3993644,407968,211445,295627,239255,443367,6162976
2,positive,105470,1493748,111888,178537,109618,48954,174532,2222747
3,sum,1580393,12116763,1002646,1049694,1178390,461214,899980,18289080


array([0.5214832 , 1.11374719, 1.28252407, 1.0145451 , 0.64231419,
       1.11037755, 0.38892293])

In [118]:
df_agg_all.groupby("lang").sum()["count"].values * (topic_tweets / all_tweets)

array([ 187503.64203146, 1437577.59439539,  118957.61159931,
        124539.55947576,  139808.52657121,   54720.12642165,
        106776.93950522])

In [134]:
# Relative percentagewise change to expected distribution in this topic to all topics
df_agg_all.query("topic==0").groupby("lang").sum()["count"].values / (
    df_agg_all.groupby("lang").sum()["count"].values * (topic_tweets / all_tweets)
)

array([0.5214832 , 1.11374719, 1.28252407, 1.0145451 , 0.64231419,
       1.11037755, 0.38892293])

In [133]:
# Percentagewise distribution of languages in this topic
df_agg_all.query("topic==0").groupby("lang").sum()["count"].values / df_agg_all.query(
    "topic==0"
).groupby("lang").sum()["count"].values.sum()

array([0.04506232, 0.73787262, 0.07031067, 0.05822938, 0.04138516,
       0.0280015 , 0.01913835])

In [132]:
# Percentagewise distribution of languages over all topics
df_agg_all.groupby("lang").sum()["count"].values / df_agg_all.groupby("lang").sum()[
    "count"
].values.sum()

array([0.08641183, 0.66251357, 0.05482211, 0.05739457, 0.06443134,
       0.025218  , 0.04920859])